In [30]:
import numpy as np
import pandas as pd 
import nltk
import re
import spacy
from gensim import corpora, models
import gensim
from gensim.matutils import hellinger
from scipy.spatial.distance import cosine
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\anaya\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [37]:
user_song = int(input("Enter song index: "))
df = pd.read_csv(r'C:\Users\anaya\OneDrive\Desktop\Anay Masters\Intro to NLP\NLP_Group_7\FullyProcessedDataset.csv')
print(user_song)
df.head(11)


5


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment
0,0,0,139419,/foo-fighters/,"Hey, Johnny Park!",/foo-fighters/hey-johnny-park.html,Come and I'll take you under\r\nThis beautiful...,en,come take beautiful bruise color everything fa...,0.8316
1,1,1,290738,/mxpx/,Call In Sick,/mxpx/call-in-sick.html,"Oh how I missed you,\r\nOh how I needed you to...",en,oh miss oh need today oh miss oh need today ca...,0.8839
2,2,2,162905,/arch-enemy/,Despicable Heroes,/arch-enemy/despicable-heroes.html,"I spit in your face, preacers and leaders\r\nS...",en,spit face preacer leader spew false dogma beli...,-0.9541
3,3,3,281035,/the-maine/,Whoever She Is,/the-maine/whoever-she-is.html,I thought I had my girl but she ran away\r\nMy...,en,think girl run away car got steal go to late w...,-0.8750
4,4,4,253213,/a-ha/,Days On End,/a-ha/days-on-end.html,Do know why winter's such a cold and lonely pl...,en,know winter cold lonely place breath bleach fa...,0.4791
5,5,5,275382,/paul-mccartney/,Teddy Boy,/paul-mccartney/teddy-boy.html,This Is A Story of A Boy Named Ted\r\nIf His M...,en,story boy name te mother say ted good would te...,0.8750
6,6,6,164954,/king-diamond/,Sorry Dear (Outro),/king-diamond/sorry-dear-outro.html,"""I wanna go home... it's so cold down here\r\n...",en,wanna go home cold like dark mommy mommiiii ev...,0.8225
7,7,7,314642,/david-bowie/,Fantastic Voyage,/david-bowie/fantastic-voyage.html,In the event that this fantastic voyage\r\nSho...,en,event fantastic voyage turn erosion never get ...,0.2349
8,8,8,287943,/donovan/,Happiness Runs,/donovan/happiness-runs.html,Little pebble upon the sand\r\nNow you're lyin...,en,little pebble upon sand lie hand many year lit...,0.9990
9,9,9,131692,/heritage-singers/,I'll Fly Away,/heritage-singers/ill-fly-away.html,Some glad morning when this life is o'er\r\nI'...,en,glad morning life er fly away home god celesti...,0.9337


In [41]:
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_score(lyrics):
    sentiment = analyzer.polarity_scores(lyrics)
    sentiment_score = sentiment['compound']
    return sentiment_score

sentiment = get_sentiment_score(df['Lemmatized_Lyrics'].iloc[user_song])
print(sentiment)

SSF_list = []
for i in range(df.shape[0]):
    SSF_list.append(abs(df['Sentiment'].iloc[i] - sentiment))

df['SSF'] = SSF_list
df.head(21)


0.875


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,SSF
0,0,0,139419,/foo-fighters/,"Hey, Johnny Park!",/foo-fighters/hey-johnny-park.html,Come and I'll take you under\r\nThis beautiful...,en,come take beautiful bruise color everything fa...,0.8316,0.0434
1,1,1,290738,/mxpx/,Call In Sick,/mxpx/call-in-sick.html,"Oh how I missed you,\r\nOh how I needed you to...",en,oh miss oh need today oh miss oh need today ca...,0.8839,0.0089
2,2,2,162905,/arch-enemy/,Despicable Heroes,/arch-enemy/despicable-heroes.html,"I spit in your face, preacers and leaders\r\nS...",en,spit face preacer leader spew false dogma beli...,-0.9541,1.8291
3,3,3,281035,/the-maine/,Whoever She Is,/the-maine/whoever-she-is.html,I thought I had my girl but she ran away\r\nMy...,en,think girl run away car got steal go to late w...,-0.8750,1.7500
4,4,4,253213,/a-ha/,Days On End,/a-ha/days-on-end.html,Do know why winter's such a cold and lonely pl...,en,know winter cold lonely place breath bleach fa...,0.4791,0.3959
5,5,5,275382,/paul-mccartney/,Teddy Boy,/paul-mccartney/teddy-boy.html,This Is A Story of A Boy Named Ted\r\nIf His M...,en,story boy name te mother say ted good would te...,0.8750,0.0000
6,6,6,164954,/king-diamond/,Sorry Dear (Outro),/king-diamond/sorry-dear-outro.html,"""I wanna go home... it's so cold down here\r\n...",en,wanna go home cold like dark mommy mommiiii ev...,0.8225,0.0525
7,7,7,314642,/david-bowie/,Fantastic Voyage,/david-bowie/fantastic-voyage.html,In the event that this fantastic voyage\r\nSho...,en,event fantastic voyage turn erosion never get ...,0.2349,0.6401
8,8,8,287943,/donovan/,Happiness Runs,/donovan/happiness-runs.html,Little pebble upon the sand\r\nNow you're lyin...,en,little pebble upon sand lie hand many year lit...,0.9990,0.1240
9,9,9,131692,/heritage-singers/,I'll Fly Away,/heritage-singers/ill-fly-away.html,Some glad morning when this life is o'er\r\nI'...,en,glad morning life er fly away home god celesti...,0.9337,0.0587


In [44]:
tfidf_vectorizer = TfidfVectorizer()
CSF_list = []
for i in range(df.shape[0]):
    lyrics_matrix = tfidf_vectorizer.fit_transform([df['Lemmatized_Lyrics'].iloc[user_song], df['Lemmatized_Lyrics'].iloc[i]])
    lyrics_similarity = cosine_similarity(lyrics_matrix)
    CSF_list.append(1 - (lyrics_similarity[0][1]))
df['CSF'] = CSF_list
df.head(11)


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,SSF,CSF
0,0,0,139419,/foo-fighters/,"Hey, Johnny Park!",/foo-fighters/hey-johnny-park.html,Come and I'll take you under\r\nThis beautiful...,en,come take beautiful bruise color everything fa...,0.8316,0.0434,9.661364e-01
1,1,1,290738,/mxpx/,Call In Sick,/mxpx/call-in-sick.html,"Oh how I missed you,\r\nOh how I needed you to...",en,oh miss oh need today oh miss oh need today ca...,0.8839,0.0089,9.658840e-01
2,2,2,162905,/arch-enemy/,Despicable Heroes,/arch-enemy/despicable-heroes.html,"I spit in your face, preacers and leaders\r\nS...",en,spit face preacer leader spew false dogma beli...,-0.9541,1.8291,1.000000e+00
3,3,3,281035,/the-maine/,Whoever She Is,/the-maine/whoever-she-is.html,I thought I had my girl but she ran away\r\nMy...,en,think girl run away car got steal go to late w...,-0.8750,1.7500,8.877081e-01
4,4,4,253213,/a-ha/,Days On End,/a-ha/days-on-end.html,Do know why winter's such a cold and lonely pl...,en,know winter cold lonely place breath bleach fa...,0.4791,0.3959,9.230182e-01
5,5,5,275382,/paul-mccartney/,Teddy Boy,/paul-mccartney/teddy-boy.html,This Is A Story of A Boy Named Ted\r\nIf His M...,en,story boy name te mother say ted good would te...,0.8750,0.0000,1.110223e-16
6,6,6,164954,/king-diamond/,Sorry Dear (Outro),/king-diamond/sorry-dear-outro.html,"""I wanna go home... it's so cold down here\r\n...",en,wanna go home cold like dark mommy mommiiii ev...,0.8225,0.0525,8.933342e-01
7,7,7,314642,/david-bowie/,Fantastic Voyage,/david-bowie/fantastic-voyage.html,In the event that this fantastic voyage\r\nSho...,en,event fantastic voyage turn erosion never get ...,0.2349,0.6401,9.776657e-01
8,8,8,287943,/donovan/,Happiness Runs,/donovan/happiness-runs.html,Little pebble upon the sand\r\nNow you're lyin...,en,little pebble upon sand lie hand many year lit...,0.9990,0.1240,9.801098e-01
9,9,9,131692,/heritage-singers/,I'll Fly Away,/heritage-singers/ill-fly-away.html,Some glad morning when this life is o'er\r\nI'...,en,glad morning life er fly away home god celesti...,0.9337,0.0587,9.709465e-01


In [ ]:
def calculate_hellinger_distance(song_dist_1, song_dist_2):
    return hellinger(song_dist_1, song_dist_2)

input_song_index = user_song

input_song_dist = song_topic_distribution[input_song_index]

hellinger_distances = []

for i, song_dist in enumerate(song_topic_distribution):

    distance = calculate_hellinger_distance(input_song_dist, song_dist)
    hellinger_distances.append((distance))

df['HDF'] = hellinger_distances

In [36]:
# for i in range(df.shape[0]):
#     if df['Cosine_Similarity'].iloc[i] >= 0.3:
#         print(df['Cosine_Similarity'].iloc[i])

df.nlargest(6,["Cosine_Similarity"])

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ALink,SName,SLink,Lyric,language,Lemmatized_Lyrics,Sentiment,Cosine_Similarity
7,7,7,314642,/david-bowie/,Fantastic Voyage,/david-bowie/fantastic-voyage.html,In the event that this fantastic voyage\r\nSho...,en,event fantastic voyage turn erosion never get ...,neu,1.000000
5020,5020,5020,140271,/soul-asylum/,You'll Live For Now,/soul-asylum/youll-live-for-now.html,Some people ain't got no choice\r\nBut we do\r...,en,people get choice people get voice wanna live ...,neu,0.363211
11272,11272,11272,135099,/the-cure/,Gone!,/the-cure/gone-2.html,Oh you know how it is\r\nwake up feeling blue\...,en,oh know wake feel blue everything could wrong ...,neu,0.334370
14962,14962,14962,186369,/the-killers/,The World We Live In,/the-killers/the-world-we-live-in.html,This is the world that we live in\r\nI feel my...,en,world live feel get tie world live well maybe ...,neu,0.315465
3533,3533,3533,304429,/bob-marley/,Why Should I,/bob-marley/why-should-i.html,Why should I bend down my head and cry?\r\nWhy...,en,bend head cry bend head cry old world end new ...,neu,0.310950
33040,33040,33041,191056,/cat-power/,Nothing But Time,/cat-power/nothing-but-time.html,"I see you, kid, alone in youExplainr room\r\nY...",en,see kid alone youexplainr room get weight mind...,neu,0.307892
